In [1]:
import pickle
from collections import namedtuple

import os
from urllib.parse import parse_qs

import ipywidgets as widgets
from ipyvuetify import VuetifyTemplate
import traitlets

Peiling = namedtuple('Peiling', ['verwacht', 'laag', 'hoog'])

In [2]:
with open('peilingen.pkl', 'rb') as fh:
    numbers = pickle.load(fh)

with open('table.pkl', 'rb') as fh:
    table = pickle.load(fh)

In [30]:
query_string = os.environ.get('QUERY_STRING', '')
server = os.environ.get('SERVER_NAME', 'localhost')
base_url = "http://" + server
port = os.environ.get('SERVER_PORT', '')
if port:
    base_url += ":" + port
base_url += os.environ.get('SCRIPT_NAME', '')
url_picked_parties = (party for party, _ in parse_qs(query_string).items() if party in numbers.keys())

# Coalitiewijzer

In [4]:
class SeatsWidget(VuetifyTemplate):
    rows = traitlets.Int(5).tag(sync=True)
    cols = traitlets.Int(30).tag(sync=True)
    low = traitlets.Int(0).tag(sync=True)
    exp = traitlets.Int(0).tag(sync=True)
    high = traitlets.Int(0).tag(sync=True)
    template = traitlets.Unicode('''
<template>
    <div>
        <table>
        <tr v-for="i in rows">
            <td v-for="j in cols/2">
                <div>
                    <v-icon v-if="((j-1) * rows + i) < low"        style="color: #666666">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) < exp"   style="color: red; opacity: 30%; transform: rotate(90deg)">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) == exp"  style="color: red">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) <= high" style="color: red; opacity: 30%; transform: rotate(-90deg)">mdi-seat</v-icon>
                    <v-icon v-else style="color: #bbbbbb; transform: rotate(180deg)">mdi-seat</v-icon>
                </div>
            </td>
            <td>
                <div>
                    <v-icon style="color: blue">mdi-dots-vertical</v-icon>
                </div>
            </td>
            <td v-for="j in cols/2">
                <div>
                    <v-icon v-if="((j + cols/2 - 1) * rows + i) < low"        style="color: #666666">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) < exp"   style="color: red; opacity: 30%; transform: rotate(90deg)">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) == exp"  style="color: red">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) <= high" style="color: red; opacity: 30%; transform: rotate(-90deg)">mdi-seat</v-icon>
                    <v-icon v-else style="color: #bbbbbb; transform: rotate(180deg)">mdi-seat</v-icon>
                </div>
            </td>
        </tr>
    </div>
</template>
''').tag(sync=True)
    
seats = SeatsWidget()

In [27]:
toggle_buttons = {key: widgets.ToggleButton(description=key, value=(key in url_picked_parties),
                                            layout=widgets.Layout(height='auto', width='auto'))
                  for key in numbers.keys()}

grid_width = 5

grid = widgets.GridspecLayout((len(numbers) - 1) // grid_width + 1, grid_width)
for ix, (key, button) in enumerate(toggle_buttons.items()):
    ix1, ix2 = ix // grid_width, ix % grid_width
    grid[ix1, ix2] = button

In [46]:
share_url = widgets.HTML(
    value=f'<a href="{base_url}">{base_url}</a>',
    description='Share link:',
)

In [42]:
def on_toggle(**toggles):
    selected = tuple(partij for partij, partij_selected in toggles.items() if partij_selected)
    peiling = table[selected]

    seats.low = peiling.laag
    seats.exp = peiling.verwacht
    seats.high = peiling.hoog
    
    url = base_url
    if len(selected) > 0:
        url = base_url + "?" + "&".join(f"{s}=1" for s in selected)

    share_url.value = f'<a href="{url}">{url}</a>'

    #     print(f"verwacht aantal zetels: {expectation_value} (laag: {low}, hoog: {high})")

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(seats)
display(grid, interact_out)

SeatsWidget(events=[], exp=76, high=79, low=73)

GridspecLayout(children=(ToggleButton(value=True, description='VVD', layout=Layout(grid_area='widget001', heig…

Output()

Druk op de knoppen om partijen toe te voegen aan je coalitie. Een meerderheid heeft 76 zetels nodig. De helder rode zetel is het verwachte aantal zetels. In licht rood (zijwaarts gekantelde zetels) zie je onder- en bovengrenzen van het verwachte aantal zetels op basis van verschillende peilingen.

Press the buttons to add parties to your coalition. A majority coalition needs 76 seats. The bright red seat is the expected number of seats. In light red (tipped sideways seats) we show the lower and upper limits of the expected amount of seats based on different polls.

In [45]:
display(share_url)

HTML(value='<a href="http://localhost">http://localhost</a>', description='Sharing link:')

---

Data: [Peilingwijzer (op basis van peilingen I&O Research, Ipsos en Kantar)](https://peilingwijzer.tomlouwerse.nl/). Code: [see blogpost on Medium](https://blog.esciencecenter.nl/coalition-polls-for-the-people-with-coalitiewijzer-68bca83b95e7).

By [Patrick Bos](https://twitter.com/egpbos/)

[<img src="logo.png" alt="Netherlands eScience Center logo" style="height: 1.5em;" align="left"/>](https://esciencecenter.nl/)